In [5]:
# to access root modules
if __name__ == "__main__" and __package__ is None:
    from sys import path
    from os.path import dirname as dir

    path.append(dir(path[0]))
    __package__ = "dash_app"

In [6]:
import pandas as pd
import numpy as np
import datetime
import dateutil
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from charts import stations_map, map_columns
from settings import *


In [7]:
data_path = DATA_PATH+'Atmospheric_Domain/2.7UpperAirTemperatureWind/Figure2.13/'
data_csv = data_path + 'Figure2.13_data.csv'

In [21]:
"""
Tidy data for charts
"""
columns_dict = {'datetime':'year',
               'mean__annual__upper_air_temperature':'temp',
               'moving_average__5year__upper_air_temperature':'av_temp5',
               'mean__annual__upper_air_wind_speed':'speed',
               'moving_average__5year__upper_air_wind_speed':'av_sp5',
                  }
xls = pd.ExcelFile(
    data_path+'UATemperatureWind_Update2012to2018.xlsx')
original_df = pd.read_excel(xls, '2Obs_ByDay')
valentia_df = map_columns(columns_dict, original_df)
valentia_df['location'] = 'Valentia'

columns_dict = {'datetime':'year',
               'mean__annual__upper_air_temperature':'temp(NewSensor)',
               'mean__annual__upper_air_wind_speed':'speed (NewSensor)',
                  }
valentia_new_df = map_columns(columns_dict, original_df)
valentia_new_df['location'] = 'Valentia (New Sensor)'
valentia_new_df

df = pd.concat([valentia_df,valentia_new_df],ignore_index=True, sort=False)
df['datetime'] = pd.to_datetime(df['datetime'],format='%Y')
#df.to_csv(data_csv) #commented out as the file should be kept as is

In [52]:
df = pd.read_csv(data_csv, index_col=0)
df

,datetime,mean__annual__upper_air_temperature,moving_average__5year__upper_air_temperature,mean__annual__upper_air_wind_speed,moving_average__5year__upper_air_wind_speed,location
0,1980-01-01,-45.888171,NaN,56.616114,NaN,Valentia
1,1981-01-01,-45.861708,NaN,51.766881,NaN,Valentia
2,1982-01-01,-45.726207,-45.797265,55.231818,52.154338,Valentia
3,1983-01-01,-45.539227,-45.877752,49.110260,51.072996,Valentia
4,1984-01-01,-45.971012,-45.868828,48.046617,51.615880,Valentia
...,...,...,...,...,...,...
77,2016-01-01,-45.159403,NaN,52.853712,NaN,Valentia (New Sensor)
78,2017-01-01,-44.873275,NaN,55.977583,NaN,Valentia (New Sensor)
79,2018-01-01,-44.627946,NaN,54.029481,NaN,Valentia (New Sensor)
80,2019-01-01,NaN,NaN,NaN,NaN,Valentia (New Sensor)


In [53]:
df = pd.read_csv(data_csv, index_col=0)
valentia_df = df[df['location']=='Valentia']
valentia_new_sensor_df = df[df['location']=='Valentia (New Sensor)']
mean_annual_trace = go.Scatter(x=valentia_df['datetime'],
                            y=valentia_df['mean__annual__upper_air_temperature'],
                         name='Annual Mean',
                         mode='markers+lines',
                         marker=dict(color=TIMESERIES_COLOR_2,
                                     size=5,
                                     opacity=0.5),
                         line=dict(color=TIMESERIES_COLOR_2,
                                      width=1),
                         hovertemplate='%{x|%Y}<br>' +
                         '<b>Annual Mean</b><br>' +
                         'Temperature: %{y:.2f} \u00b0C<br>' +
                         '<extra></extra>'
                         )
moving_avg_trace = go.Scatter(x=valentia_df['datetime'],
                            y=valentia_df['moving_average__5year__upper_air_temperature'],
                            name='5 Year Moving Average',
                            mode='lines',  # 'line' is default
                            line_shape='spline',
                            line=dict(color=TIMESERIES_COLOR_1,
                                      width=2),
                            hovertemplate='%{x|%Y}<br>' +
                            '<b>5yr Moving Average</b><br>' +
                            'Temperature: %{y:.2f} \u00b0C<extra></extra>'
                            )

mean_annual_trace_new_sensor = go.Scatter(x=valentia_new_sensor_df['datetime'],
                            y=valentia_new_sensor_df['mean__annual__upper_air_temperature'],
                         name='Annual Mean (New Sensor)',
                         mode='markers+lines',
                         marker=dict(color=TIMESERIES_COLOR_2,
                                     size=5,
                                     opacity=0.5),
                        line=dict(color=TIMESERIES_COLOR_2,
                                  dash='dash',
                                      width=1),
                         hovertemplate='%{x|%Y}<br>' +
                         '<b>Annual Mean</b><br>' +
                         'Temperature: %{y:.2f} \u00b0C<br>' +
                         '<extra></extra>'
                         )

In [54]:
figure_2_13_a = go.Figure(data=[mean_annual_trace, mean_annual_trace_new_sensor,moving_avg_trace], layout=TIMESERIES_LAYOUT)
figure_2_13_a.update_layout(
    yaxis=dict(title='Temperature at 300 hPa (\u00b0C)'),
    xaxis=dict(title="Year"))
figure_2_13_a